<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/In_class_exercise/In_class_exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (20 points in total)**

# 1. Text Data Preprocessing

Here is a [legal case](https://github.com/unt-iialab/INFO5731_FALL2020/blob/master/In_class_exercise/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



## 1.1 Basic feature extraction using text data (4 points)

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

## 1.2 Basic Text Pre-processing of text data (4 points)

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

## 1.3 Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above. (4 points)


## 1.4 Advance Text Processing (Extra credit: 4 points)

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.


In [1]:
# Write your code here

############################################################################################################################

# 1.1 BASIC FEATURE EXTRACTION USING TEXT DATA

file = r"C:\Users\Raheyma Arshad\Desktop\Legal Case - Class Exercise 03.txt"
f = open(file, encoding="utf8")
text = f.read().splitlines()
while("" in text) : 
    text.remove("") 

# Number of sentences
number_of_sentences = len(text)
print('The number of sentences is:', number_of_sentences)

# Number of words
word_count = len(str(text).split(" "))
print ('The number of words is:', word_count)

# Number of characters
character_count = sum(len(i) for i in text)
print ('The number of characters is:', character_count)

# Average word length
words = []
for sentence in text:
    words.append(sentence.split())

words_list = []
for sublist in words:
    for item in sublist:
        words_list.append(item)

each_word_length = []
for word in words_list:
    each_word_length.append(len(word))

avg_word_length = sum(each_word_length)/len(each_word_length)
print ('The average word length is:', avg_word_length)

# Number of stopwords
from nltk.corpus import stopwords
import nltk
stop = stopwords.words('english')

stopwords = []
for word in words_list:
    if word in stop:
        stopwords.append(word)
        
number_of_stopwords = len(stopwords)
print ('The number of stopwords is:', number_of_stopwords)

# Number of special characters
special_characters = []
for word in words_list:
    if word.startswith('*'):
        special_characters.append(word)
        
number_of_special_characters = len(special_characters)
print ('The number of special characters is:', number_of_special_characters)

# Number of numerics
numerics = []
for word in words_list:
    if word.isdigit():
        numerics.append(word)
        
number_of_numerics = len(numerics)
print ('The number of numerics is:', number_of_numerics)

# Number of uppercase words
uppercase = []
for word in words_list:
    if word.isupper():
        uppercase.append(word)
        
number_of_uppercase = len(uppercase)
print ('The number of uppercase words is:', number_of_uppercase)

The number of sentences is: 148
The number of words is: 3722
The number of characters is: 20295
The average word length is: 4.510385756676558
The number of stopwords is: 1679
The number of special characters is: 13
The number of numerics is: 60
The number of uppercase words is: 84


In [2]:
############################################################################################################################

# 1.2 BASIC TEXT PRE-PROCESSING OF TEXT DATA

import pandas as pd
import numpy as np

data = pd.DataFrame(text)
data.columns = ['text']
data['text'].replace('', np.nan, inplace=True)
data.dropna(subset=['text'], inplace=True)

# Lower casing
data['text'] = data['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# Punctuation removal
data['text'] = data['text'].str.replace('[^\w\s]','')

# Stopwords removal
data['text'] = data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# Frequent words removal
high_freq = pd.Series(' '.join(data['text']).split()).value_counts()[:10]
high_freq = list(high_freq.index)
data['text'] = data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in high_freq))

# Rare words removal
low_freq = pd.Series(' '.join(data['text']).split()).value_counts()[-10:]
low_freq = list(low_freq.index)
data['text'] = data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in low_freq))

# Spelling correction
from textblob import TextBlob
data['text'] = data['text'].apply(lambda x: str(TextBlob(x).correct()))

# Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
data['text'] = data['text'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# Lemmatization
from textblob import Word
data['text'] = data['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

data['text'].replace('', np.nan, inplace=True)
data.dropna(subset=['text'], inplace=True)

# Tokenization
data['text'] = data['text'].apply(lambda x: nltk.word_tokenize(x))

data['text'].head()

0         [5, all, 740]
1     [suprem, alabama]
2                [adam]
4      [manner, norton]
5    [june, term, 1843]
Name: text, dtype: object

In [3]:
############################################################################################################################

# 1.3 SAVE ALL THE CLEAN SENTENCES TO A CSV FILE 

data.to_csv(r"C:\Users\Raheyma Arshad\Desktop\Clean Sentences.csv", index=False)

In [4]:
############################################################################################################################

# 1.4 ADVANCE TEXT PROCESSING

# Calculate the term frequency of all the terms.
term_freq = (data['text']).apply(lambda x: pd.value_counts(x)).sum(axis = 0).reset_index()
term_freq.columns = ['words', 'tf']
print(term_freq)

# Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.
data['text_1'] = [','.join(map(str, l)) for l in data['text']]

# Top 10 1-gram
from nltk.util import ngrams
data['1-grams'] = data['text'].apply(lambda row: list(nltk.ngrams(row, 1)))
gram_1 = list(set([a for b in data['1-grams'].tolist() for a in b]))

gram_1_counter = {}
for gram in gram_1:
    if gram in gram_1_counter:
        gram_1_counter[gram] += 1
    else:
        gram_1_counter[gram] = 1

sorted_gram_1 = sorted(gram_1_counter, key = gram_1_counter.get, reverse = True)
top_10_gram_1 = sorted_gram_1[:10]

print('The top 10 1-gram are:')
print(top_10_gram_1)

# Top 10 2-grams
data['2-grams'] = data['text'].apply(lambda row: list(nltk.ngrams(row, 2)))
gram_2 = list(set([a for b in data['2-grams'].tolist() for a in b]))

gram_2_counter = {}
for gram in gram_2:
    if gram in gram_2_counter:
        gram_2_counter[gram] += 1
    else:
        gram_2_counter[gram] = 1

sorted_gram_2 = sorted(gram_2_counter, key = gram_2_counter.get, reverse = True)
top_10_gram_2 = sorted_gram_2[:10]

print('The top 10 2-gram are:')
print(top_10_gram_2)

# Top 10 3-grams
data['3-grams'] = data['text'].apply(lambda row: list(nltk.ngrams(row, 3)))
gram_3 = list(set([a for b in data['3-grams'].tolist() for a in b]))

gram_3_counter = {}
for gram in gram_3:
    if gram in gram_3_counter:
        gram_3_counter[gram] += 1
    else:
        gram_3_counter[gram] = 1

sorted_gram_3 = sorted(gram_3_counter, key = gram_3_counter.get, reverse = True)
top_10_gram_3 = sorted_gram_3[:10]

print('The top 10 3-gram are:')
print(top_10_gram_3)


       words    tf
0        all  17.0
1          5   8.0
2        740   2.0
3    alabama   1.0
4     suprem   1.0
..       ...   ...
700      rye   1.0
701     half   1.0
702     file   2.0
703      neg   2.0
704  histori   2.0

[705 rows x 2 columns]
The top 10 1-gram are:
[('firm',), ('condit',), ('major',), ('seen',), ('wm',), ('meant',), ('singular',), ('inclin',), ('writ',), ('point',)]
The top 10 2-gram are:
[('746', 'seen'), ('statut', 'design'), ('septemb', 'control'), ('confirm', 'correct'), ('sold', 'assumpt'), ('point', 'inquiri'), ('all', '464'), ('attach', 'upon'), ('fact', 'state'), ('offic', 'levi')]
The top 10 3-gram are:
[('act', '1821', 'ask'), ('power', 'take', 'posse'), ('u', 'back', 'question'), ('cotton', 'count', 'appeal'), ('bill', 'except', 'seal'), ('must', 'may', '1840'), ('upon', 'subject', 'polici'), ('defend', 'might', 'time'), ('accord', 'evid', 'ad'), ('learn', 'comment', 'doubt')]


# 2. Python Regular Expression

## 2.1 Write a Python program to remove leading zeros from an IP address. (4 points)

ip = "260.08.094.109"

In [5]:
# Write your code here

import re

def remove_zeros(ip): 
    new_ip = re.sub('\.[0]*', '.', ip)
    return new_ip
    
      
remove_zeros("260.09.094.109")

'260.9.94.109'

## 2.2 Write a Python Program to extract all the years from the following sentence. (4 points)

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [6]:
# Write your code here

import re

def extract_years(text):
    # Find all four digit numbers
    four_digit_numbers = re.findall(r'(\d{4})', text)
    # Convert them to integers
    four_digit_numbers = [int(i) for i in four_digit_numbers] 
    # Remove numbers below 1500 because usually they do not represent years 
    for i in four_digit_numbers:
        if i < 1500:
            four_digit_numbers.remove(i)
    # Remove any duplicates
    years = list(dict.fromkeys(four_digit_numbers))
    return years
        
sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

extract_years(sentence)

[2010, 2019]